In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
df=pd.read_csv("datasets/cardio_train.csv", delimiter=';')
data=pd.read_csv("datasets/cardio_train.csv")
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [42]:
features = ['id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio']

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


In [5]:
X = df.drop(['cardio'], axis=1)
y= df['cardio']

from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
from sklearn.tree import DecisionTreeClassifier, export_text

In [30]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

grid = GridSearchCV(DecisionTreeClassifier(random_state=42), params, cv=5)
grid.fit(X_train, y_train)

best_clf = grid.best_estimator_
print("Best Parameters:", grid.best_params_)


Best Parameters: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 2}


In [36]:
dt = DecisionTreeClassifier(min_samples_split = 7, random_state=42, max_depth = 5, criterion = 'gini')
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
acc_dt = accuracy_score(y_test, y_pred_dt)
print(f"Decision Tree Accuracy: {acc_dt:.4f}")

Decision Tree Accuracy: 0.7344


In [37]:
print(classification_report(y_test, y_pred_dt))


              precision    recall  f1-score   support

           0       0.71      0.79      0.75      6988
           1       0.76      0.68      0.72      7012

    accuracy                           0.73     14000
   macro avg       0.74      0.73      0.73     14000
weighted avg       0.74      0.73      0.73     14000



In [38]:
print(confusion_matrix(y_test, y_pred_dt))


[[5507 1481]
 [2237 4775]]


In [12]:
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [13]:
y_pred_rf=rf.predict(X_test)
y_pred_rf

array([1, 1, 1, ..., 0, 1, 1])

In [32]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.72      0.79      0.75      6988
           1       0.77      0.69      0.73      7012

    accuracy                           0.74     14000
   macro avg       0.74      0.74      0.74     14000
weighted avg       0.74      0.74      0.74     14000



In [15]:
print(confusion_matrix(y_test, y_pred_rf))

[[5527 1461]
 [2192 4820]]


In [17]:
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

In [18]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
y_pred_cv = best_model.predict(X_test)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END bootstrap=True, max_depth=40, max_fea

/home/abhishek/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
115 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
51 fits failed with the following error:
Traceback (most recent call last):
  File "/home/abhishek/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/abhishek/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/home/abhishek/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/base.py", line 436, in _validate_para

In [19]:
print(classification_report(y_test, y_pred_cv))
print(confusion_matrix(y_test, y_pred_cv))

              precision    recall  f1-score   support

           0       0.72      0.79      0.75      6988
           1       0.77      0.69      0.73      7012

    accuracy                           0.74     14000
   macro avg       0.74      0.74      0.74     14000
weighted avg       0.74      0.74      0.74     14000

[[5497 1491]
 [2155 4857]]


In [20]:
from sklearn.tree import DecisionTreeClassifier,export_text

dt_surrogate = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_surrogate.fit(X_train, best_model.predict(X_train))

rules = export_text(dt_surrogate, feature_names=list(X.columns))
print(rules)

|--- ap_hi <= 129.50
|   |--- cholesterol <= 2.50
|   |   |--- age <= 22203.50
|   |   |   |--- ap_lo <= 98.00
|   |   |   |   |--- ap_lo <= 89.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- ap_lo >  89.50
|   |   |   |   |   |--- class: 0
|   |   |   |--- ap_lo >  98.00
|   |   |   |   |--- age <= 19828.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- age >  19828.50
|   |   |   |   |   |--- class: 1
|   |   |--- age >  22203.50
|   |   |   |--- ap_lo <= 78.00
|   |   |   |   |--- active <= 0.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- active >  0.50
|   |   |   |   |   |--- class: 0
|   |   |   |--- ap_lo >  78.00
|   |   |   |   |--- active <= 0.50
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- active >  0.50
|   |   |   |   |   |--- class: 0
|   |--- cholesterol >  2.50
|   |   |--- age <= 19317.50
|   |   |   |--- gluc <= 2.50
|   |   |   |   |--- ap_lo <= 72.50
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- ap_lo >  72.50
|   |   

In [47]:
from sklearn.tree import _tree

def get_linear_rules(tree, feature_names, node_index=0, current_rule="IF ", rules=[]):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    if tree_.feature[node_index] != _tree.TREE_UNDEFINED:
        name = feature_name[node_index]
        threshold = tree_.threshold[node_index]

        # Left child rule
        left_rule = f"{current_rule}{name} <= {threshold:.2f} THEN "
        get_linear_rules(tree, feature_names, tree_.children_left[node_index], left_rule, rules)

        # Right child rule
        right_rule = f"{current_rule}{name} > {threshold:.2f} THEN "
        get_linear_rules(tree, feature_names, tree_.children_right[node_index], right_rule, rules)
    else:
        value = tree_.value[node_index]
        predicted_class = value.argmax()
        samples = int(tree_.n_node_samples[node_index])
        rules.append(f"{current_rule}PREDICT class {predicted_class} [samples: {samples}]")

    return rules

# Get linear rules
linear_rules = get_linear_rules(dt_surrogate, features)

# Save to file
with open("results/cardio_rules.txt", "w") as f:
    for rule in linear_rules:
        f.write(rule + "\n")

print("Linear decision rules saved to results/cardio_rules_dt.txt")


Linear decision rules saved to results/cardio_rules_dt.txt


In [22]:
pip install xgboost

  Using cached xgboost-2.1.4-py3-none-manylinux_2_28_x86_64.whl.metadata (2.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 MB 4.6 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 MB 2.2 MB/s eta 0:00:0000:0100:05
Note: you may need to restart the kernel to use updated packages.


In [23]:
import xgboost
from xgboost import XGBClassifier
xg=XGBClassifier()
xg.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [24]:
y_pred_xg=xg.predict(X_test)

print(classification_report(y_test, y_pred_xg))

              precision    recall  f1-score   support

           0       0.72      0.77      0.74      6988
           1       0.75      0.70      0.72      7012

    accuracy                           0.73     14000
   macro avg       0.73      0.73      0.73     14000
weighted avg       0.73      0.73      0.73     14000



In [25]:
param_dist2 = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 5, 7, 9, 11],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_lambda': [0.1, 1.0, 10.0],
    'reg_alpha': [0, 0.1, 1.0]
}

In [26]:
random_search2 = RandomizedSearchCV(
    estimator=xg,
    param_distributions=param_dist2,
    n_iter=50,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

In [27]:
random_search2.fit(X_train, y_train)

best_xg = random_search2.best_estimator_
y_pred_xgcv = best_xg.predict(X_test)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.3, max_depth=3, min_child_weight=3, n_estimators=300, reg_alpha=0, reg_lambda=0.1, subsample=0.8; total time=   1.5s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.3, max_depth=3, min_child_weight=3, n_estimators=300, reg_alpha=0, reg_lambda=0.1, subsample=0.8; total time=   1.6s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.3, max_depth=3, min_child_weight=3, n_estimators=300, reg_alpha=0, reg_lambda=0.1, subsample=0.8; total time=   1.8s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.3, max_depth=3, min_child_weight=3, n_estimators=300, reg_alpha=0, reg_lambda=0.1, subsample=0.8; total time=   1.7s
[CV] END colsample_bytree=1.0, gamma=0, learning_rate=0.3, max_depth=3, min_child_weight=3, n_estimators=300, reg_alpha=1.0, reg_lambda=1.0, subsample=0.8; total time=   1.8s
[CV] END colsample_bytree=1.0, gamma=0, learning_rate=0.3, max_depth=3,

In [28]:
print("Best parameters:", random_search2.best_params_)
print(classification_report(y_test, y_pred_xgcv))
print(confusion_matrix(y_test, y_pred_xgcv))

Best parameters: {'subsample': 1.0, 'reg_lambda': 10.0, 'reg_alpha': 0.1, 'n_estimators': 300, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.6}
              precision    recall  f1-score   support

           0       0.72      0.78      0.75      6988
           1       0.76      0.70      0.73      7012

    accuracy                           0.74     14000
   macro avg       0.74      0.74      0.74     14000
weighted avg       0.74      0.74      0.74     14000

[[5417 1571]
 [2100 4912]]
